1.Importe cet ensemble de données de tweets dans un DataFrame.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

In [ ]:
df = pd.read_csv('https://github.com/murpi/wilddata/raw/master/quests/tweets.zip')
df


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


2.Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?

In [ ]:
df=df[df['sentiment']!="neutral"]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16363 entries, 1 to 27478
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         16363 non-null  object
 1   text           16363 non-null  object
 2   selected_text  16363 non-null  object
 3   sentiment      16363 non-null  object
dtypes: object(4)
memory usage: 639.2+ KB


In [ ]:
df['sentiment'].value_counts(normalize=True)*100

positive    52.447595
negative    47.552405
Name: sentiment, dtype: float64

3.Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.

In [ ]:
X = df["text"]
y = df["sentiment"]

X_train,X_test,y_train, y_test = train_test_split(X, y, random_state=32, train_size = 0.75)

print("The length of the initial dataset is :", len(X))
print("The length of the train dataset is   :", len(X_train))
print("The length of the test dataset is    :", len(X_test))

The length of the initial dataset is : 16363
The length of the train dataset is   : 12272
The length of the test dataset is    : 4091


4.Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.

In [ ]:
# Modèle vectorizer

vectorizer = CountVectorizer()

vectorizer.fit(X_train, y_train)

CountVectorizer()

In [ ]:
# Entraine modèle sur X_train  


X_train_CV = vectorizer.transform(X_train)

In [ ]:
X_train_CV

<12272x15806 sparse matrix of type '<class 'numpy.int64'>'
	with 144578 stored elements in Compressed Sparse Row format>

In [ ]:
# Entraine modèle sur X_test 

X_test_CV = vectorizer.transform(X_test)

In [ ]:
X_test_CV 

<4091x15806 sparse matrix of type '<class 'numpy.int64'>'
	with 44633 stored elements in Compressed Sparse Row format>

5.Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et 0.877 pour l'ensemble de test.

In [ ]:
from sklearn.linear_model import LogisticRegression
modelLR = LogisticRegression()

In [ ]:
modelLR.fit(X_train_CV,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred_train = modelLR.predict(X_train_CV)
y_pred_test = modelLR.predict(X_test_CV)

In [ ]:
print(metrics.accuracy_score(y_train,y_pred_train))
print(metrics.accuracy_score(y_test,y_pred_test))

0.9663461538461539
0.8772916157418724


6.Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

In [ ]:
pd.DataFrame(data = metrics.confusion_matrix(y_test, y_pred_test),
             index = modelLR.classes_ + " actual",
             columns = modelLR.classes_ + " predicted")

,negative predicted,positive predicted
negative actual,1700,235
positive actual,267,1889


In [ ]:
X_test

5680      - no,  is buttfuck stupid. I`m just silly and...
7661      get better omg i still dont believe that i di...
2670     HollowbabesHere comes the utter shite #bgt <I ...
5020      Thank You Clayton. Going to my favorite Greek...
26962     I`m watching it at the moment  -sighs- and st...
                               ...                        
4062                                       I can`t take it
4618      so where r u spinning now that the Hookah is ...
18293              WHAT?! i was wanting to see that show!!
16606                     Har vondt i ryggen My back hurts
5223     Laying in bed with a book & some beautiful mus...
Name: text, Length: 4091, dtype: object

In [ ]:
y_test

5680     negative
7661     negative
2670     negative
5020     positive
26962    negative
           ...   
4062     negative
4618     negative
18293    negative
16606    negative
5223     positive
Name: sentiment, Length: 4091, dtype: object

In [ ]:
y_pred_test

array(['negative', 'negative', 'positive', ..., 'negative', 'negative',
       'positive'], dtype=object)

In [ ]:
df_test_concat= pd.concat([X_test, y_test],axis=1)

In [ ]:
df_test_concat

,text,sentiment
5680,"- no, is buttfuck stupid. I`m just silly and...",negative
7661,get better omg i still dont believe that i di...,negative
2670,HollowbabesHere comes the utter shite #bgt <I ...,negative
5020,Thank You Clayton. Going to my favorite Greek...,positive
26962,I`m watching it at the moment -sighs- and st...,negative
...,...,...
4062,I can`t take it,negative
4618,so where r u spinning now that the Hookah is ...,negative
18293,WHAT?! i was wanting to see that show!!,negative
16606,Har vondt i ryggen My back hurts,negative


In [ ]:
df_test_concat["predict_sentiment"] = y_pred_test
df_test_concat["predict_sentiment"]

5680     negative
7661     negative
2670     positive
5020     positive
26962    negative
           ...   
4062     negative
4618     negative
18293    negative
16606    negative
5223     positive
Name: predict_sentiment, Length: 4091, dtype: object

In [ ]:
error_pred = df_test_concat.loc[df_test_concat["sentiment"] != df_test_concat["predict_sentiment"]]

In [ ]:
error_pred.info() # total FN+FP=502

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502 entries, 2670 to 12007
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               502 non-null    object
 1   sentiment          502 non-null    object
 2   predict_sentiment  502 non-null    object
dtypes: object(3)
memory usage: 15.7+ KB


In [ ]:
error_pred.sample(10)  # sample pour donner le résultat aléatoir.

,text,sentiment,predict_sentiment
2872,Youtube`s tubes appear to be clogging up somew...,negative,positive
2443,"Yeah, Sabrina..though I keep thinking it`s so...",negative,positive
13262,I`m feeling better now that I have something i...,positive,negative
6133,You really are always dirty!!! Ha ha!!!! Do y...,negative,positive
10769,Bella doesn`t love me no more,negative,positive
8730,victory for the bulldogs was celebrated by 3 w...,negative,positive
2667,NO WAY! i didn`t know he was singing ROF that...,positive,negative
8351,I`m jumping off the nearest bridge if I can`t ...,negative,positive
11678,Summer Glau to appear in Dollhouse next year h...,negative,positive
20105,wishes people wouldn`t be so stubborn sometimes.,positive,negative
